<font size=+3 color="navy"><center><b>COVID-19 Pandemic in Greece: Cases, Deaths, and Vaccination Progress</b></center></font>

<img src="https://images.unsplash.com/photo-1555993539-1732b0258235?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1050&q=80" width = 700>
<center><em>Photo by Spencer Davis (Unsplash)</em></center>

<br>

This notebook aims to provide an overview of the COVID-19 pandemic in <font color="dodgerblue"><b>Greece</b></font>. Specifically, it explores:
-	The current situation and how the pandemic has evolved since its beginning.
-	How vaccinations are progressing in the country.

This notebook is by no means a complete analysis. Its goal is to paint the general picture of COVID-19 in Greece through various visualisations. It is written so that it can be reused for any other country, provided that the country's name exists in all five datasets. 

In the [GitHub repository](https://github.com/KOrfanakis/COVID-19_in_Greece) of this project, I have included a PowerPoint presentation with a summary of all figures along with some of my insights.

<br>

**Table of Contents**

- [Libraries](#Libraries)
- [Default Parameters](#Default-Parameters)
- [Functions](#Functions)
    - [customise_layout()](#customise_layout())<br>
    - [daily_covid_progress()](#daily_covid_progress())<br>
    - [create_intervals()](#create_intervals())<br>
    - [create_map_intervals()](#create_map_intervals())<br>
    - [create_waffle()](#create_waffle())<br>
- [Data](#Data)
    - [Summary Dataset](#Summary-Dataset)<br>
    - [Daily Dataset](#Daily-Dataset)<br>
    - [Tests Dataset](#Tests-Dataset)<br>
    - [Vaccination Dataset](#Vaccination-Dataset)<br>
    - [Government Response Tracker](#Government-Response-Tracker)
- [What's the Current State of the Pandemic?](#What's-the-Current-State-of-the-Pandemic?)
- [How has the Pandemic Progressed?](#How-has-the-Pandemic-Progressed?)
    - [Confirmed Cases](#Confirmed-Cases)<br>
    - [Deaths](#Deaths)<br>
    - [Tests](#Tests)
- [The Case Fatality Rate](#The-Case-Fatality-Rate)
- [Vaccination Overview](#Vaccination-Overview)
- [Vaccination Progress](#Vaccination-Progress)
- [Oxford's Stringency Index](#Extra:-Oxford's-Stringency-Index)
- [Extra Resources](#Extra-Resources)

<br>

# Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime, timedelta

from matplotlib.colors import rgb2hex
from plotly.subplots import make_subplots

!pip install pywaffle
from pywaffle import Waffle

In [ ]:
print('✔️ Libraries imported!')

<br>

# Default Parameters

In this section, we will define the country of interest (in our case, Greece) as a variable called `country`. Additionally, we will set some defaults parameters for the whole notebook. These parameters are related to figures created with Matplotlib and Plotly.

In [ ]:
country = 'Greece'
print('Country: ', country)

In [ ]:
default_font = 'sans serif'
default_color = 'white'
paper_bgcolor = 'black'
font_size = 18
height, width = 500, 850  # Default dimensions for all Plotly graph objects.

plt.rcParams['font.family'] = ['serif']
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'black'

%config InlineBackend.figure_format = 'retina'

print('✔️ Default parameters set!')

<br>

# Functions

Since we are going to reuse parts of the code, it will be helpful to define some functions.

## `customise_layout()`

In [ ]:
def customise_layout(fig,
                     height,
                     width,
                     linewidth=1.5,
                     linecolor=default_color,
                     font_size=font_size,
                     font_color=default_color,
                     default_font=default_font):
    '''Customise the layout for Plotly objects by setting the default colors, dimensions, axis parameters, etc.'''
    fig.update_layout(xaxis=dict(linewidth=linewidth,
                                 linecolor=linecolor,
                                 color=linecolor,
                                 ticks='inside',
                                 tickwidth=2,
                                 ticklen=5,
                                 showgrid=False,
                                 mirror=True),
                      yaxis=dict(linewidth=linewidth,
                                 linecolor=linecolor,
                                 showgrid=False,
                                 mirror=True),
                      font=dict(size=font_size,
                                color=font_color,
                                family=default_font),
                      title=dict(font=dict(size=font_size + 10)),
                      paper_bgcolor=paper_bgcolor,
                      plot_bgcolor=paper_bgcolor,
                      height=height,
                      width=width)


print('✔️ Function defined!')

## `daily_covid_progress()`

In [ ]:
def daily_covid_progress(fig, df, country, type_, color):
    '''Create a Plotly figure showing the progress of COVID-19 cases or deaths for one country.'''
    # Define the data.
    df_country = df[df['Country'].isin([country])]

    # Add the two traces
    fig.add_trace(go.Bar(x=df_country['Date'],
                         y=df_country['Daily New {}'.format(type_)],
                         name='New {}'.format(type_),
                         marker=dict(line=dict(color=color, width=1)),
                         marker_color=color),
                  secondary_y=False)

    fig.add_trace(go.Scatter(x=df_country['Date'],
                             y=df_country['Cumulative Total {}'.format(type_)],
                             name='Cumulative No.',
                             line=dict(color='red', width=4)),
                  secondary_y=True)

    # Update the axes parameters
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(showgrid=False, secondary_y=False)
    fig.update_yaxes(title_text='Cumulative number of {}'.format(type_.lower()),
                     color='red',
                     rangemode='tozero',
                     showgrid=False,
                     secondary_y=True)

    # Update the layout parameters
    title = 'Daily new confirmed COVID-19 {} in {}'.format(type_.lower(), country)
    fig.update_layout(title={'text': title,
                             'y': 0.925},
                      hovermode='x unified',
                      showlegend=False)

    return fig


print('✔️ Function defined!')

## `create_intervals()`

In [ ]:
def create_intervals(df, column, intervals, cmap):
    '''Prepare a Pandas DataFrame for creating a map (with discrete colours instead of continuous colourmaps).'''
    df_copy = df.copy()
    df_copy['Intervals'] = pd.cut(df_copy[column],
                                  intervals,
                                  include_lowest=True)
    df_copy.sort_values(by='Intervals', inplace=True)

    cmap = plt.get_cmap(cmap, df_copy['Intervals'].nunique() + 4)
    colors = [rgb2hex(cmap(i)) for i in range(cmap.N)]

    color_map = dict(zip(df_copy['Intervals'].value_counts(sort=False).index.values.astype(str), colors))
    color_map.update({'nan': 'gray'})

    df_copy['Intervals'] = df_copy['Intervals'].astype(str)

    return color_map, df_copy


print('✔️ Function defined!')

## `create_map_intervals()`

In [ ]:
def create_map_intervals(df, column, intervals, cmap, title, showlegend=False):
    '''Create a Plotly map with discrete colors corresponding to the different intervals in the data.'''
    color_map, df = create_intervals(df,
                                     column=column,
                                     intervals=intervals,
                                     cmap=cmap)

    fig = px.choropleth(df,
                        locations='Country',
                        locationmode='country names',
                        color='Intervals',
                        fitbounds='locations',
                        color_discrete_map=color_map,
                        hover_name='Country',
                        hover_data={
                            'Intervals': False,
                            'Country': False,
                            column: ':.2f'
                        })

    fig.update_layout(title={'text': title,
                             'y': 0.95},
                      geo=dict(landcolor='gray',
                               showlakes=False,
                               bgcolor='rgba(0,0,0,0)'),
                      margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
    fig.update_traces(showlegend=showlegend)
    fig.update_geos(resolution=50, visible=False)

    fig.add_annotation(x=0.05,
                       y=0.90,
                       xref='paper',
                       yref='paper',
                       text='Shown is the rolling 7-day average.',
                       showarrow=False,
                       font=dict(family='sans serif',
                                 size=font_size,
                                 color='white'),
                       align='center',
                       bordercolor=None)

    return fig


print('✔️ Function defined!')

## `create_waffle()`

In [ ]:
def create_waffle(values, colors, icon, label):
    '''Create a simple waffle plot to visualise the percentage of people vaccinated (either partially or fully).'''
    fig = plt.figure(FigureClass=Waffle,
                     rows=5,
                     columns=20,
                     figsize=(14, 6),
                     values=values,
                     colors=colors,
                     icons=icon,
                     labels=[f'{k} ({v:.1f}%)' for k, v in values.items()],
                     legend={
                         'loc': 'lower left',
                         'labelcolor': 'white',
                         'bbox_to_anchor': (0.255, +0.93),
                         'ncol': 2,
                         'frameon': False,
                         'fontsize': 23
                     },
                     title={
                         'label': label,
                         'weight': 'bold',
                         'loc': 'center',
                         'fontdict': {'fontsize': 32},
                         'color': 'white'
                     })

    plt.axvline(x=1.01, color='white', lw=3, ls='--')
    plt.axvline(x=1.01 * 2 + 0.02, color='white', lw=3, ls='--')
    plt.axvline(x=1.01 * 3 + 0.04, color='white', lw=3, ls='--')


print('✔️ Function defined!')

<br>

# Data

Our analysis requires five datasets.

## Summary Dataset

In [ ]:
summary_df = pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv')
summary_df.columns = [col.replace('_', ' ').title() for col in summary_df.columns]

print('✔️ Dataset imported!\n')
print('It contains {} rows x {} columns.'.format(summary_df.shape[0],
                                                 summary_df.shape[1]))
print('Sample of three rows:')
summary_df.sample(3)

## Daily Dataset

In [ ]:
daily_df = pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv',
                       parse_dates=['date'])
daily_df.columns = [col.replace('_', ' ').title() for col in daily_df.columns]

print('✔️ Dataset imported!\n')
print('It contains {} rows x {} columns.'.format(daily_df.shape[0],
                                                 daily_df.shape[1]))
print('\nFirst date:', daily_df['Date'].min().strftime('%Y-%m-%d'))
print(' Last date:', daily_df['Date'].max().strftime('%Y-%m-%d'))
print('\nLast 3 rows:')
daily_df.tail(3)

## Tests Dataset

In [ ]:
tests_df = pd.read_csv(
    r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv',
    parse_dates=['date'],
    usecols=[
        'location', 'date', 'new_tests', 'total_tests',
        'total_tests_per_thousand', 'new_tests_per_thousand',
        'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
        'positive_rate', 'tests_per_case', 'tests_units'
    ])
tests_df.columns = [col.replace('_', ' ').title() for col in tests_df.columns]
tests_df.rename({'Location': 'Country'}, axis=1, inplace=True)

print('✔️ Dataset imported!\n')
print('It contains {} rows x {} columns.'.format(tests_df.shape[0],
                                                 tests_df.shape[1]))
print('\nFirst date:', tests_df['Date'].min().strftime('%Y-%m-%d'))
print(' Last date:', tests_df['Date'].max().strftime('%Y-%m-%d'))
print('\nLast 3 rows:')
tests_df.tail(3)

## Vaccination Dataset

In [ ]:
vacc_df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv',
                      parse_dates=['date'])
vacc_df.columns = [col.replace('_', ' ').title() for col in vacc_df.columns]

print('✔️ Dataset imported!\n')
print('It contains {} rows x {} columns.'.format(vacc_df.shape[0],
                                                 vacc_df.shape[1]))
print('\nFirst date:', vacc_df['Date'].min().strftime('%Y-%m-%d'))
print(' Last date:', vacc_df['Date'].max().strftime('%Y-%m-%d'))
print('\nLast 3 rows:')
vacc_df.tail(3)

## Government Response Tracker

In [ ]:
rt_df = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv',
                    usecols=['CountryName', 'RegionName', 'Date', 'ConfirmedCases', 'StringencyIndex'],
                    dtype=object,
                    parse_dates=['Date'])
rt_df.columns = ['Country', 'Region', 'Date', 'Confirmed Cases', 'Stringency Index']

rt_df['Region'] = rt_df['Region'].replace(np.nan, '-')
rt_df['Full Name'] = np.where(rt_df['Region'] != '-',
                              rt_df['Country'] + ' - ' + rt_df['Region'],
                              rt_df['Country'])

print('✔️ Dataset imported!\n')
print('It contains {} rows x {} columns.'.format(rt_df.shape[0],
                                                 rt_df.shape[1]))
print('\nFirst date:', rt_df['Date'].min().strftime('%Y-%m-%d'))
print(' Last date:', rt_df['Date'].max().strftime('%Y-%m-%d'))
print('\nLast 3 rows:')
rt_df.tail(3)

Unfortunately, the datasets are not updated on the same day. To deal with this issue, we will first find the last date common in all datasets. Then, we will select all entries before that date for all datasets. 

In [ ]:
dates_list = []
for df in [daily_df, vacc_df, tests_df, rt_df]:
    dates_list.append(df[df['Country'] == country]['Date'].max())

cutoff_date = min(dates_list).strftime('%Y-%m-%d')
print('The last date that is common in all datasets: ', cutoff_date)

daily_df = daily_df[daily_df['Date'] <= min(dates_list)]
tests_df = tests_df[tests_df['Date'] <= min(dates_list)]
vacc_df = vacc_df[vacc_df['Date'] <= min(dates_list)]
rt_df = rt_df[rt_df['Date'] <= min(dates_list)]

print('✔️ Datasets modified!')

<br>

# What's the Current State of the Pandemic?

In [ ]:
summary = summary_df[summary_df['Country'] == country]
population = summary['Population'].values[0]

features = ['Total Confirmed', 'Total Recovered', 'Total Deaths', 'Active Cases', 'Population']
headers = ['Cases (total)', 'Recovered', 'Deaths', 'Active', 'Population']

fig, ax = plt.subplots(1, len(features), figsize=(17, 3))

title = '{} - Summary (as of {})'.format(country, cutoff_date)
ax[0].text(0.02, 0.8, title, size=30, color='white', weight='bold')

for index, feature in enumerate(features):
    ax[index].text(0.5,
                   0.5,
                   headers[index],
                   ha='center',
                   va='center',
                   fontsize=23,
                   color='white',
                   weight='heavy',
                   bbox=dict(edgecolor='w', facecolor='steelblue', pad=6))

    ax[index].text(0.5,
                   0.2,
                   '{:,}'.format(int(summary[feature])),
                   ha='center',
                   va='center',
                   fontsize=30,
                   fontweight='bold',
                   color='white')

    ax[index].set_axis_off()

<br>

# How has the Pandemic Progressed?

## Confirmed Cases

In [ ]:
fig = daily_covid_progress(fig=make_subplots(specs=[[{'secondary_y': True}]]),
                           df=daily_df,
                           country=country,
                           type_='Cases',
                           color='steelblue')

customise_layout(fig, height=height, width=width)
fig.show()

In [ ]:
one_week_date = datetime.strptime(cutoff_date, '%Y-%m-%d').date() - timedelta(days=7)
one_week_date = one_week_date.strftime('%Y-%m-%d')

daily_7mean = daily_df[daily_df['Date'] > one_week_date].groupby('Country')
daily_7mean = daily_7mean[['Daily New Cases', 'Daily New Deaths']].mean()
daily_7mean.reset_index(inplace=True)

daily_7mean = pd.merge(daily_7mean,
                       summary_df[['Country', 'Population']],
                       left_on='Country',
                       right_on='Country')

daily_7mean['Daily New Cases (per M) - 7day Avg'] = 1 * 1E+6 * daily_7mean['Daily New Cases'] / daily_7mean['Population']
daily_7mean['Daily New Deaths (per M) - 7day Avg'] = 1 * 1E+6 * daily_7mean['Daily New Deaths'] / daily_7mean['Population']

fig = create_map_intervals(
    daily_7mean,
    column='Daily New Cases (per M) - 7day Avg',
    intervals=[0, 0.5, 2.5, 5, 10, 50, 100, 250, 500, 1000, float('inf')],
    cmap='Reds',
    title='Daily new cases per million people ({})'.format(cutoff_date),
    showlegend=False)

customise_layout(fig, width=width, height=height)
fig.update_layout(paper_bgcolor='black')

fig.show()

<br>

## Deaths

In [ ]:
fig = daily_covid_progress(fig=make_subplots(specs=[[{'secondary_y': True}]]),
                           df=daily_df,
                           country=country,
                           type_='Deaths',
                           color='paleturquoise')

customise_layout(fig, height=height, width=width)
fig.show()

In [ ]:
fig = create_map_intervals(
    daily_7mean.dropna(),
    column='Daily New Deaths (per M) - 7day Avg',
    intervals=[0, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, float('inf')],
    cmap='Reds',
    title='Daily new deaths per million people ({})'.format(cutoff_date),
    showlegend=False)

customise_layout(fig, width=width, height=height)
fig.update_layout(paper_bgcolor='black')

fig.show()

<br>

## Tests

By looking at the number of tests, we can determine if big spikes in cases/deaths are real or just a result of changes in the number of tests.

In [ ]:
colors = ['steelblue', 'darkorange']

# Define the data
daily_country = daily_df[daily_df['Country'].isin([country])]
tests_country = tests_df[tests_df['Country'].isin([country])]

# Add the traces
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=daily_country['Date'],
                         y=daily_country['Daily New Cases'].rolling(7).mean(),
                         mode='lines',
                         line=dict(color=colors[0], width=3),
                         name='New Cases'),
              secondary_y=False)

fig.add_trace(go.Scatter(x=tests_country['Date'],
                         y=tests_country['New Tests'].rolling(7).mean().interpolate(),
                         mode='lines',
                         line=dict(color=colors[1], width=3),
                         name='New Tests'),
              secondary_y=True)

# Update the axes parameters
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Daily New Cases',
                 color=colors[0],
                 linecolor=colors[0],
                 linewidth=1.5,
                 showgrid=False,
                 rangemode='tozero',
                 secondary_y=False)
fig.update_yaxes(title_text='Daily New Tests',
                 color=colors[1],
                 linecolor=colors[1],
                 linewidth=1.5,
                 showgrid=False,
                 rangemode='tozero',
                 secondary_y=True)

# Update the layout parameters
fig.update_layout(title={'text': 'Daily Cases and Tests in {}'.format(country),
                         'y': 0.95},
                  hovermode='x unified',
                  showlegend=False)

fig.add_annotation(x=-0.06,
                   y=1.15,
                   xref='paper',
                   yref='paper',
                   text='Shown is the rolling 7-day average.',
                   showarrow=False,
                   font=dict(family='sans serif',
                             size=font_size,
                             color='white'),
                   align='center',
                   bordercolor=None)

customise_layout(fig, height=height, width=width)

fig.show()

<br>

# The Case Fatality Rate

The case fatality rate is the number of *confirmed* deaths divided by the number of *confirmed* cases ([Ref. 1](#Extra-Resources)). For more information on this rate and how to interpret it correctly, you can read more [here](https://ourworldindata.org/covid-mortality-risk).

In [ ]:
df_country = daily_df[daily_df['Country'] == country]

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_country['Date'],
               y=df_country['Cumulative Total Deaths'] / df_country['Cumulative Total Cases'],
               line=dict(color='dodgerblue', width=4),
               fill='tozeroy',
               fillcolor='rgba(135, 206, 250, 0.2)'))

# Update the axes parameters
fig.update_xaxes(title_text='Date')
fig.update_yaxes(rangemode='tozero', tickformat=',.2%')

# Update the layout parameters
title = 'The case fatality rate in {}'.format(country)
fig.update_layout(title={'text': title,
                         'y': 0.925},
                  hovermode='x unified',
                  showlegend=False)

customise_layout(fig, height=height, width=width)

fig.show()

<br>

# Vaccination Overview

In [ ]:
vaccinations = vacc_df[vacc_df['Country'] == country].iloc[-1]
vaccination_scheme = vaccinations['Vaccines']
features = ['Total Vaccinations', 'People Vaccinated', 'People Fully Vaccinated', 'Daily Vaccinations']
labels = [' Total Vaccinations ', ' People Vaccinated \n(at least 1 dose)', ' People Fully \nVaccinated', ' Daily Vaccinations ']

fig, ax = plt.subplots(1, len(features), figsize=(17, 4))

ax[0].text(0.02,
           0.85,
           '{} (as of {})'.format(country, cutoff_date),
           size=30,
           color='white',
           weight='bold')
ax[0].text(0.02, 0.7, '    Vaccines: ', size=21, color='white', weight='bold')
ax[0].text(0.75,
           0.7,
           '{}'.format(vaccination_scheme),
           size=21,
           color='darkorange',
           weight='bold')

for index, feature in enumerate(features):
    ax[index].text(0.5,
                   0.5,
                   labels[index],
                   ha='center',
                   va='center',
                   fontsize=19,
                   color='white',
                   weight='bold',
                   bbox=dict(edgecolor='white', facecolor='steelblue', pad=5))

    if feature != 'Daily Vaccinations':
        norm = vaccinations[feature + ' Per Hundred']
        text = '%'
    else:
        norm = vaccinations[feature + ' Per Million']
        text = '\nper million'

    ax[index].text(0.5,
                   0.2,
                   '{:,}\n({:.1f}{})'.format(int(vaccinations[feature]), norm, text),
                   ha='center',
                   va='center',
                   fontsize=25,
                   fontweight='bold',
                   color='white')

    ax[index].set_axis_off()

<br>

We can use a [waffle plot](https://github.com/gyli/PyWaffle) to visualise the percentage of the population that has been vaccinated.

In [ ]:
First_dose = {'Υes': vaccinations['People Vaccinated Per Hundred'],
              'No': 100 - vaccinations['People Vaccinated Per Hundred']}

Second_dose = {'Yes': vaccinations['People Fully Vaccinated Per Hundred'],
               'No': 100 - vaccinations['People Fully Vaccinated Per Hundred']}

create_waffle(values=First_dose,
              colors=['darkorange', 'darkgray'],
              icon='user',
              label='Vaccinated (One Dose)\n')

create_waffle(values=Second_dose,
              colors=['tomato', 'darkgray'],
              icon='user',
              label='Fully Vaccinated\n')

<br>

# Vaccination Progress

In [ ]:
features = ['Total Vaccinations', 'People Vaccinated', 'People Fully Vaccinated']
labels = ['Total Vaccinations', 'People Vaccinated (One dose)', 'People Fully Vaccinated']
colors = ['bisque', 'lightskyblue', 'lightcoral']
fillcolors = ['rgba(255, 228, 196, 0.2)', 'rgba(135, 206, 250, 0.2)', 'rgba(240, 128, 128, 0.2)']

# Define the data
df_country = vacc_df[vacc_df['Country'].isin([country])]

# Add the traces
fig = go.Figure()

for index, feature in enumerate(features):
    fig.add_trace(
        go.Scatter(x=df_country['Date'],
                   y=df_country[feature].interpolate(),
                   mode='lines',
                   name=labels[index],
                   line=dict(color=colors[index], width=4),
                   fill='tozeroy',
                   fillcolor=fillcolors[index]))

# Update the axes parameters
fig.update_xaxes(title_text='Date')
fig.update_yaxes(rangemode='tozero')

# Update the layout parameters
fig.update_layout(
    title={'text': 'Vaccination Progress in {}'.format(country),
           'y': 0.98},
    hovermode='x unified',
    showlegend=True,
    legend=dict(orientation='h',
                xanchor='left',
                x=0.02,
                yanchor='bottom',
                y=1.01))

customise_layout(fig, height=height, width=width)

fig.show()

In [ ]:
features = ['Daily Vaccinations', 'Total Vaccinations']
colors = ['bisque', 'red']

# Define the data
df_country = vacc_df[vacc_df['Country'].isin([country])]

# Add the traces
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Bar(x=df_country['Date'],
                     y=df_country[features[0]].interpolate(),
                     marker_color=colors[0],
                     marker=dict(line=dict(color=colors[0], width=1)),
                     name=features[0]),
              secondary_y=False)

fig.add_trace(go.Scatter(x=df_country['Date'],
                         y=df_country[features[1]].interpolate(),
                         mode='lines',
                         line=dict(color=colors[1], width=3),
                         name=features[1]),
              secondary_y=True)

# Update the axes parameters
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text=features[0],
                 color=colors[0],
                 linecolor=colors[0],
                 linewidth=1.5,
                 showgrid=False,
                 secondary_y=False)
fig.update_yaxes(title_text=features[1],
                 color=colors[1],
                 linecolor=colors[1],
                 linewidth=1.5,
                 showgrid=False,
                 rangemode='tozero',
                 secondary_y=True)

# Update the layout parameters
fig.update_layout(title={'text': 'Daily Vaccinations in {}'.format(country),
                         'y': 0.925},
                  hovermode='x unified',
                  showlegend=False)

customise_layout(fig, height=height, width=width)

fig.show()

In [ ]:
col = 'People Fully Vaccinated Per Hundred'
vacc_groupby = vacc_df.groupby('Country', dropna=False)[col].last().to_frame().reset_index()

fig = px.choropleth(vacc_groupby,
                    locations='Country',
                    locationmode='country names',
                    color=col,
                    fitbounds='locations',
                    color_continuous_scale='Reds',
                    hover_name='Country',
                    hover_data={
                        'Country': False,
                        'People Fully Vaccinated Per Hundred': ':.1f'
                    })

fig.update_coloraxes(showscale=False)
fig.update_layout(title={'text': 'Share of the population fully vaccinated against COVID-19' +
                                 '<br>({})'.format(cutoff_date),
                         'y':0.93},
                  geo=dict(landcolor='gray',
                           showlakes=False,
                           bgcolor='rgba(0,0,0,0)'),
                  margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
fig.update_geos(resolution=50, visible=False)

customise_layout(fig, width=width, height=height)

fig.show()

<br>

# Extra: Oxford's Stringency Index

To understand how governments have responded to the pandemic, we rely on data from the [Oxford Coronavirus Government Response Tracker](https://www.bsg.ox.ac.uk/research/research-projects/covid-19-government-response-tracker) (OxCGRT), which is published and managed by researchers at the Blavatnik School of Government at the University of Oxford.

For our analysis, we will use the "**[Stringency Index](https://www.bsg.ox.ac.uk/research/research-projects/covid-19-government-response-tracker)**", which records the strictness of "lockdown style" policies that primarily restrict people’s behaviour (e.g. school closures, workplace closures, and travel bans).

In [ ]:
# Create a new dataframe with only the country of interest
df_sel = rt_df[rt_df['Country'] == country]

# Plot the country's strigency index over time
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_sel['Date'],
               y=df_sel['Stringency Index'].astype(float),
               name='Value',
               mode='lines+markers',
               marker=dict(size=4, color='dodgerblue'),
               fill='tozeroy',
               fillcolor='rgba(135, 206, 250, 0.2)'))

# Update the axes parameters
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Stringency Index', range=[0, 100])

# Update the layout parameters
fig.update_layout(title={'text': "Oxford's Stringency Index in {}".format(country),
                         'y': 0.925},
                  showlegend=False,
                  hovermode='x unified')

customise_layout(fig, height=height, width=width)

fig.show()

In [ ]:
colors = ['dodgerblue', 'bisque']
df_sel = rt_df[rt_df['Date'] == cutoff_date]
hovertemplate = df_sel['Full Name'] + '<br><br>Index: %{y:.2f}' + '<br>Cases: %{x}'

# Add the trace
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_sel['Confirmed Cases'].astype(float),
               y=df_sel['Stringency Index'].astype(float),
               mode='markers',
               marker=dict(size=np.where(df_sel['Country'] == country, 14, 7),
                           color=np.where(df_sel['Country'] == country, colors[0], colors[1]),
                           line=dict(color=np.where(df_sel['Country'] == country, 'white', 'lavender'), width=1),
                           opacity=np.where(df_sel['Country'] == country, 1, 0.4)),
               name='',
               hovertemplate=hovertemplate))

# Update the axes parameters
fig.update_xaxes(title_text='Confirmed Cases', type='log')
fig.update_yaxes(title_text='Stringency Index', range=[0, 100])

# Update the layout parameters
title = "Oxford's Stringency Index - Comparison ({})".format(cutoff_date)
fig.update_layout(title={'text': title, 'y': 0.925}, showlegend=False)

customise_layout(fig, height=height, width=width)

fig.show()

**Note**: By using the cumulative number of confirmed cases as the independent variable, we do not take into account the population of each country/region. In a future version, I may try normalising per million people, which might mean keeping only country data and discarding data on individual regions.

<br>

# Extra Resources

The main source of inspiration for some of the figures included in this notebook was:

1. [Greece: Coronavirus Pandemic Country Profile](https://ourworldindata.org/coronavirus/country/greece) from [Our World in Data](https://ourworldindata.org/).

Additional resources are:

2. [Daily reports on COVID](https://eody.gov.gr/en/category/covid-19-en/) from the [Greek National Public Health Organization](https://eody.gov.gr/en/).
3. [COVID-19 vaccination statistics](https://www.data.gov.gr/datasets/mdg_emvolio/) from [data.gov.gr](https://www.data.gov.gr/).
4. [COVID-19 pandemic in Greece](https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Greece) from [Wikipedia](https://en.wikipedia.org/wiki/Main_Page).

<br>

Our notebook came to an end! If you liked its content, please consider <font size=+0 color="#DF0000"><b>upvoting</b></font>. <font size=+0 color="green"><b>Suggestions</b></font> are always welcome. 🙂